In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [2]:
import polars as pl
df = pl.read_csv("DataSet_1/train.csv")
df.head()

id,date,store_nbr,item_nbr,unit_sales,onpromotion
i64,str,i64,i64,f64,str
0,"""2013-01-01""",25,103665,7.0,null
1,"""2013-01-01""",25,105574,1.0,null
2,"""2013-01-01""",25,105575,2.0,null
3,"""2013-01-01""",25,108079,1.0,null
4,"""2013-01-01""",25,108701,1.0,null


#### Checking if on promotion is useful

In [3]:
df['onpromotion'].unique()

onpromotion
str
"""True"""
null
"""False"""


In [4]:
df.columns

['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']

In [5]:
# Convert 'onpromotion' to binary:
# - "True" -> 1
# - "False" and null -> 0
df = df.with_columns(
    (pl.when(pl.col('onpromotion') == "True")
       .then(1)
       .otherwise(0)
     ).alias('is_onpromotion')
)

# Calculate Pearson correlation
correlation = df.select([
    pl.corr('is_onpromotion', 'unit_sales').alias('correlation')
])

print(correlation)

shape: (1, 1)
┌─────────────┐
│ correlation │
│ ---         │
│ f64         │
╞═════════════╡
│ 0.053606    │
└─────────────┘


In [6]:
df["is_onpromotion"].unique()

is_onpromotion
i32
0
1


In [7]:
df = df.drop("onpromotion")

In [8]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion
i64,str,i64,i64,f64,i32
0,"""2013-01-01""",25,103665,7.0,0
1,"""2013-01-01""",25,105574,1.0,0
2,"""2013-01-01""",25,105575,2.0,0
3,"""2013-01-01""",25,108079,1.0,0
4,"""2013-01-01""",25,108701,1.0,0


## merging holiday.csv

In [9]:
# Load holidays.csv
holidays_df = pl.read_csv('DataSet_1/holidays_events.csv')
holidays_df.head()


date,type,locale,locale_name,description,transferred
str,str,str,str,str,bool
"""2012-03-02""","""Holiday""","""Local""","""Manta""","""Fundacion de Manta""",false
"""2012-04-01""","""Holiday""","""Regional""","""Cotopaxi""","""Provincializacion de Cotopaxi""",false
"""2012-04-12""","""Holiday""","""Local""","""Cuenca""","""Fundacion de Cuenca""",false
"""2012-04-14""","""Holiday""","""Local""","""Libertad""","""Cantonizacion de Libertad""",false
"""2012-04-21""","""Holiday""","""Local""","""Riobamba""","""Cantonizacion de Riobamba""",false


In [10]:
holidays_df['type'].unique()

type
str
"""Holiday"""
"""Event"""
"""Work Day"""
"""Additional"""
"""Transfer"""
"""Bridge"""


#### creating a new column with binary ---> is holiday or not (1,0)..

In [11]:
# Merge with holidays_df on 'date'
df = df.join(holidays_df[['date', 'type', 'locale', 'locale_name']], on='date', how='left')

# Create a new column 'is_holiday' (1 if holiday, 0 otherwise)
df = df.with_columns(
    (pl.col('type') == "Holiday").cast(pl.Int8).alias('is_holiday')
)

# Print the first few rows to verify
print(df.head())

shape: (5, 10)
┌─────┬────────────┬───────────┬──────────┬───┬─────────┬──────────┬─────────────┬────────────┐
│ id  ┆ date       ┆ store_nbr ┆ item_nbr ┆ … ┆ type    ┆ locale   ┆ locale_name ┆ is_holiday │
│ --- ┆ ---        ┆ ---       ┆ ---      ┆   ┆ ---     ┆ ---      ┆ ---         ┆ ---        │
│ i64 ┆ str        ┆ i64       ┆ i64      ┆   ┆ str     ┆ str      ┆ str         ┆ i8         │
╞═════╪════════════╪═══════════╪══════════╪═══╪═════════╪══════════╪═════════════╪════════════╡
│ 0   ┆ 2013-01-01 ┆ 25        ┆ 103665   ┆ … ┆ Holiday ┆ National ┆ Ecuador     ┆ 1          │
│ 1   ┆ 2013-01-01 ┆ 25        ┆ 105574   ┆ … ┆ Holiday ┆ National ┆ Ecuador     ┆ 1          │
│ 2   ┆ 2013-01-01 ┆ 25        ┆ 105575   ┆ … ┆ Holiday ┆ National ┆ Ecuador     ┆ 1          │
│ 3   ┆ 2013-01-01 ┆ 25        ┆ 108079   ┆ … ┆ Holiday ┆ National ┆ Ecuador     ┆ 1          │
│ 4   ┆ 2013-01-01 ┆ 25        ┆ 108701   ┆ … ┆ Holiday ┆ National ┆ Ecuador     ┆ 1          │
└─────┴────────────┴─────

In [13]:
df.columns

['id',
 'date',
 'store_nbr',
 'item_nbr',
 'unit_sales',
 'is_onpromotion',
 'type',
 'locale',
 'locale_name',
 'is_holiday']

In [14]:
df = df.drop(['type', 'locale', 'locale_name'])

In [16]:
df.columns

['id',
 'date',
 'store_nbr',
 'item_nbr',
 'unit_sales',
 'is_onpromotion',
 'is_holiday']

In [15]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday
i64,str,i64,i64,f64,i32,i8
0,"""2013-01-01""",25,103665,7.0,0,1
1,"""2013-01-01""",25,105574,1.0,0,1
2,"""2013-01-01""",25,105575,2.0,0,1
3,"""2013-01-01""",25,108079,1.0,0,1
4,"""2013-01-01""",25,108701,1.0,0,1


## merging transaction.csv

In [17]:
transactions_df = pl.read_csv('DataSet_1/transactions.csv')

In [18]:
transactions_df.head()

date,store_nbr,transactions
str,i64,i64
"""2013-01-01""",25,770
"""2013-01-02""",1,2111
"""2013-01-02""",2,2358
"""2013-01-02""",3,3487
"""2013-01-02""",4,1922


In [19]:
transactions_df

date,store_nbr,transactions
str,i64,i64
"""2013-01-01""",25,770
"""2013-01-02""",1,2111
"""2013-01-02""",2,2358
"""2013-01-02""",3,3487
"""2013-01-02""",4,1922
…,…,…
"""2017-08-15""",50,2804
"""2017-08-15""",51,1573
"""2017-08-15""",52,2255


In [20]:
df = df.join(transactions_df[['date', 'store_nbr', 'transactions']], on=['date', 'store_nbr'], how='left')

In [21]:
df = df.with_columns(
    pl.col('transactions').fill_null(0)
)

In [22]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions
i64,str,i64,i64,f64,i32,i8,i64
0,"""2013-01-01""",25,103665,7.0,0,1,770
1,"""2013-01-01""",25,105574,1.0,0,1,770
2,"""2013-01-01""",25,105575,2.0,0,1,770
3,"""2013-01-01""",25,108079,1.0,0,1,770
4,"""2013-01-01""",25,108701,1.0,0,1,770


In [36]:
def optimize_dtypes(df):
    """Optimizes data types of a Polars DataFrame, using unsigned ints where possible."""

    for col in df.columns:
        if df[col].dtype == pl.Utf8 or df[col].dtype == pl.Boolean:  # Skip strings and booleans
            continue

        min_val = df[col].min()
        max_val = df[col].max()

        if df[col].dtype == pl.Int8 or df[col].dtype == pl.Int16 or df[col].dtype == pl.Int32 or df[col].dtype == pl.Int64:
            if min_val >= 0:  # Check for non-negative values - Use Unsigned if possible
                if max_val < 2**8:
                    df = df.with_columns(pl.col(col).cast(pl.UInt8))
                elif max_val < 2**16:
                    df = df.with_columns(pl.col(col).cast(pl.UInt16))
                elif max_val < 2**32:
                    df = df.with_columns(pl.col(col).cast(pl.UInt32))
            else:  # Signed integer if negative values are present
                if min_val >= -2**7 and max_val < 2**7:
                    df = df.with_columns(pl.col(col).cast(pl.Int8))
                elif min_val >= -2**15 and max_val < 2**15:
                    df = df.with_columns(pl.col(col).cast(pl.Int16))
                elif min_val >= -2**31 and max_val < 2**31:
                    df = df.with_columns(pl.col(col).cast(pl.Int32))

        elif df[col].dtype == pl.UInt8 or df[col].dtype == pl.UInt16 or df[col].dtype == pl.UInt32 or df[col].dtype == pl.UInt64:
            if max_val < 2**8:
                df = df.with_columns(pl.col(col).cast(pl.UInt8))
            elif max_val < 2**16:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))
            elif max_val < 2**32:
                df = df.with_columns(pl.col(col).cast(pl.UInt32))

        elif df[col].dtype == pl.Float32 or df[col].dtype == pl.Float64:
            if min_val >= np.finfo(np.float32).min and max_val <= np.finfo(np.float32).max:
                df = df.with_columns(pl.col(col).cast(pl.Float32))  # Cast to Float32 if possible

    return df

In [37]:
type(df)

polars.dataframe.frame.DataFrame

In [38]:
df = optimize_dtypes(df)


In [39]:
df

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions
u32,str,u8,u32,f32,u8,u8,u16
0,"""2013-01-01""",25,103665,7.0,0,1,770
1,"""2013-01-01""",25,105574,1.0,0,1,770
2,"""2013-01-01""",25,105575,2.0,0,1,770
3,"""2013-01-01""",25,108079,1.0,0,1,770
4,"""2013-01-01""",25,108701,1.0,0,1,770
…,…,…,…,…,…,…,…
125497035,"""2017-08-15""",54,2089339,4.0,0,1,802
125497036,"""2017-08-15""",54,2106464,1.0,1,1,802
125497037,"""2017-08-15""",54,2110456,192.0,0,1,802


##### Saving the file

In [40]:
df.write_parquet('merged_file_optimized_dtypes.paraquet')


## Feature Engineering

In [41]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [42]:
import numpy as np
import polars as pl

In [43]:
df = pl.read_parquet('merged_file_optimized_dtypes.paraquet')

In [46]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions
u32,str,u8,u32,f32,u8,u8,u16
0,"""2013-01-01""",25,103665,7.0,0,1,770
1,"""2013-01-01""",25,105574,1.0,0,1,770
2,"""2013-01-01""",25,105575,2.0,0,1,770
3,"""2013-01-01""",25,108079,1.0,0,1,770
4,"""2013-01-01""",25,108701,1.0,0,1,770


In [44]:
df.shape

(127970257, 8)

In [51]:
df.estimated_size

<bound method DataFrame.estimated_size of shape: (127_970_257, 8)
┌───────────┬────────────┬───────────┬──────────┬────────────┬────────────┬────────────┬───────────┐
│ id        ┆ date       ┆ store_nbr ┆ item_nbr ┆ unit_sales ┆ is_onpromo ┆ is_holiday ┆ transacti │
│ ---       ┆ ---        ┆ ---       ┆ ---      ┆ ---        ┆ tion       ┆ ---        ┆ ons       │
│ u32       ┆ str        ┆ u8        ┆ u32      ┆ f32        ┆ ---        ┆ u8         ┆ ---       │
│           ┆            ┆           ┆          ┆            ┆ u8         ┆            ┆ u16       │
╞═══════════╪════════════╪═══════════╪══════════╪════════════╪════════════╪════════════╪═══════════╡
│ 0         ┆ 2013-01-01 ┆ 25        ┆ 103665   ┆ 7.0        ┆ 0          ┆ 1          ┆ 770       │
│ 1         ┆ 2013-01-01 ┆ 25        ┆ 105574   ┆ 1.0        ┆ 0          ┆ 1          ┆ 770       │
│ 2         ┆ 2013-01-01 ┆ 25        ┆ 105575   ┆ 2.0        ┆ 0          ┆ 1          ┆ 770       │
│ 3         ┆ 2013-01-01 

In [52]:
# Convert 'date' column to datetime
df = df.with_columns(pl.col('date').str.strptime(pl.Date).alias('date'))

In [53]:
# 2. Handle missing values (check and fill if needed)
df = df.with_columns(
    pl.col('unit_sales').fill_null(0)
)

In [54]:
df = df.with_columns([
    pl.col('date').dt.day().alias('day'),
    pl.col('date').dt.month().alias('month'),
    pl.col('date').dt.weekday().alias('weekday'),
    pl.col('date').dt.year().alias('year'),
])

In [55]:
# 1. Log transformation of 'unit_sales' using log1p (log(1 + x))
df = df.with_columns(
    (pl.col('unit_sales') + 1).log().alias('log_unit_sales')
)

In [56]:
# 2. Z-score normalization of 'log_unit_sales' (if needed)
mean_unit_sales = df.select(pl.col('log_unit_sales').mean()).to_numpy()[0][0]
std_unit_sales = df.select(pl.col('log_unit_sales').std()).to_numpy()[0][0]


In [57]:
df = df.with_columns(
    ((pl.col('log_unit_sales') - mean_unit_sales) / std_unit_sales).alias('normalized_unit_sales')
)

In [58]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales,normalized_unit_sales
u32,date,u8,u32,f32,u8,u8,u16,i8,i8,i8,i32,f32,f32
0,2013-01-01,25,103665,7.0,0,1,770,1,1,2,2013,2.079442,NaN
1,2013-01-01,25,105574,1.0,0,1,770,1,1,2,2013,0.693147,NaN
2,2013-01-01,25,105575,2.0,0,1,770,1,1,2,2013,1.098612,NaN
3,2013-01-01,25,108079,1.0,0,1,770,1,1,2,2013,0.693147,NaN
4,2013-01-01,25,108701,1.0,0,1,770,1,1,2,2013,0.693147,NaN


In [59]:
#df = df.drop('id')
df = df.drop('normalized_unit_sales')

In [60]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales
u32,date,u8,u32,f32,u8,u8,u16,i8,i8,i8,i32,f32
0,2013-01-01,25,103665,7.0,0,1,770,1,1,2,2013,2.079442
1,2013-01-01,25,105574,1.0,0,1,770,1,1,2,2013,0.693147
2,2013-01-01,25,105575,2.0,0,1,770,1,1,2,2013,1.098612
3,2013-01-01,25,108079,1.0,0,1,770,1,1,2,2013,0.693147
4,2013-01-01,25,108701,1.0,0,1,770,1,1,2,2013,0.693147


In [61]:
# Check for nulls in each column
df.select([pl.col(c).is_null().sum().alias(c) for c in df.columns])


id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,106142049,0,0,0,0,0,0


In [63]:
df = df.with_columns(
    pl.col("is_holiday").fill_null(0)
)

In [64]:
# Check for nulls in each column
df.select([pl.col(c).is_null().sum().alias(c) for c in df.columns])


id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
df.columns

['id',
 'date',
 'store_nbr',
 'item_nbr',
 'unit_sales',
 'is_onpromotion',
 'is_holiday',
 'transactions',
 'day',
 'month',
 'weekday',
 'year',
 'log_unit_sales']

In [66]:
df.head()

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales
u32,date,u8,u32,f32,u8,u8,u16,i8,i8,i8,i32,f32
0,2013-01-01,25,103665,7.0,0,1,770,1,1,2,2013,2.079442
1,2013-01-01,25,105574,1.0,0,1,770,1,1,2,2013,0.693147
2,2013-01-01,25,105575,2.0,0,1,770,1,1,2,2013,1.098612
3,2013-01-01,25,108079,1.0,0,1,770,1,1,2,2013,0.693147
4,2013-01-01,25,108701,1.0,0,1,770,1,1,2,2013,0.693147


In [67]:
df=optimize_dtypes(df)

In [68]:
df

id,date,store_nbr,item_nbr,unit_sales,is_onpromotion,is_holiday,transactions,day,month,weekday,year,log_unit_sales
u32,date,u8,u32,f32,u8,u8,u16,u8,u8,u8,u16,f32
0,2013-01-01,25,103665,7.0,0,1,770,1,1,2,2013,2.079442
1,2013-01-01,25,105574,1.0,0,1,770,1,1,2,2013,0.693147
2,2013-01-01,25,105575,2.0,0,1,770,1,1,2,2013,1.098612
3,2013-01-01,25,108079,1.0,0,1,770,1,1,2,2013,0.693147
4,2013-01-01,25,108701,1.0,0,1,770,1,1,2,2013,0.693147
…,…,…,…,…,…,…,…,…,…,…,…,…
125497035,2017-08-15,54,2089339,4.0,0,1,802,15,8,2,2017,1.609438
125497036,2017-08-15,54,2106464,1.0,1,1,802,15,8,2,2017,0.693147
125497037,2017-08-15,54,2110456,192.0,0,1,802,15,8,2,2017,5.262691


In [70]:
df.write_parquet("processed_sales_data_optimized_dtypes.parquet")
